In [1]:
import re
import string
import json
import openai
import nltk
import base64
import random
import requests
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words
import concurrent.futures
from lyricsgenius import Genius
import time
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/nalinkrishnan/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
genius_access_token = 'wjybx3D8JdeyaQkkoezVikd3B2YhexZHEiJRcFXi6pVafExahHJmvEN2PioP36F8'
genius_secret_key = b'FUB9WH4hUIiLxFFgofyaPuQVjLYPJ1YARk-CFfHHkAYA5EKFUsaK7EVzJGdeTEdY3dcfi0zemgv5_C5IGWM7Qw'

In [1]:
file_path = 'songlabels.txt'
file_contents = ""

with open(file_path, 'r') as file:
    for line in file:
        file_contents += line

print(len(file_contents))
file_contents = file_contents.replace('"', "'")
split_list = file_contents.split('), ')
data = []
for idx, element in enumerate(split_list):
    title, sep, artist = element.partition("', '")
    title = title[2:]
    if idx != len(split_list) - 1:
        artist = artist[:-1]
    else:
        artist = artist[:-3]
    title = title.strip()
    title = title.rstrip()
    artist = artist.strip()
    artist = artist.rstrip()
    data.append((title, artist))

for song in data[-25:]:
    print(song)

print(len(data))

596667
('Hurricane', 'The Band Of Heathens')
('Todos Hablan, Nada Saben', 'Aldo Trujillo')
('UNAPOLOGETICALLY COUNTRY AS HELL', 'HARDY')
('Honey Bee', 'Blake Shelton')
('American Honey', 'Lady A')
('House On Fire', 'Bailey Zimmerman')
("Can't Help Falling in Love", 'Haley Reinhart')
('CAREFUL', 'NF')
('WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil Wayne) - Remix', 'Jack Harlow')
('HANDSOMER (Remix) (Feat. Ktlyn)', 'Russ')
('Broke Boys', 'Drake')
('Pink Houses', 'John Mellencamp')
('HEAVEN AND BACK', 'Chase Atlantic')
('HIGHEST IN THE ROOM', 'Travis Scott')
('my tears ricochet', 'Taylor Swift')
('Reflections', 'The Neighbourhood')
("I Ain't Worried", 'OneRepublic')
('Ribs', 'Lorde')
('Bad Romance', 'Lady Gaga')
("Why'd You Only Call Me When You're High?", 'Arctic Monkeys')
('Son Of A Sinner', 'Jelly Roll')
('Rumor', 'Lee Brice')
('BESO', 'ROSALÍA')
('WATATI (feat. Aldo Ranks) [From Barbie The Album]', 'KAROL G')
('Rosyln', 'Bon Iver')
15853


In [ ]:
def get_artist_id(name):
    # Step 2: Get an access token using Client Credentials Flow
    token_request_url = "https://accounts.spotify.com/api/token"
    CLIENT_ID = 'f7dcab53528747688bad9a44952c150a'
    CLIENT_SECRET = 'c7d690f022104c43a61d6a5ca1c76a81'

    BASE64_ENCODED_HEADER_STRING = base64.b64encode(bytes(f"{CLIENT_ID}:{CLIENT_SECRET}", "ISO-8859-1")).decode("ascii")
    auth_headers = {}
    auth_data = {}

    auth_headers['Content-Type'] = 'application/x-www-form-urlencoded'

    auth_data['grant_type'] = "client_credentials"
    auth_data['client_id'] = CLIENT_ID
    auth_data['client_secret'] = CLIENT_SECRET

    auth_response = requests.post(url=token_request_url, headers=auth_headers, data=auth_data)
    token = ''
    if auth_response.status_code == 200:
        print(json.dumps(auth_response.json(), indent=2)) 
        token = auth_response.json()['access_token']
    else:
        print('Error: Unable to obtain access token')
        return None
    
    artist_id = ''
    search_url = f'https://api.spotify.com/v1/search?q={name}&type=artist'
    headers = {'Authorization': 'Bearer {token}'.format(token=token)}
    response = requests.get(url=search_url, headers=headers)
    if response.status_code == 200:
        artist_data = response.json()['artists']['items'][0]
        artist_id = artist_data['id']
    else:
        print(response.status_code)
        print('Error: Unable to fetch artist data')
    return artist_id

In [ ]:
def get_artist_genres(songs, artist_ids):
    token_request_url = "https://accounts.spotify.com/api/token"
    CLIENT_ID = 'f7dcab53528747688bad9a44952c150a'
    CLIENT_SECRET = 'c7d690f022104c43a61d6a5ca1c76a81'

    BASE64_ENCODED_HEADER_STRING = base64.b64encode(bytes(f"{CLIENT_ID}:{CLIENT_SECRET}", "ISO-8859-1")).decode("ascii")
    auth_headers = {}
    auth_data = {}

    auth_headers['Content-Type'] = 'application/x-www-form-urlencoded'

    auth_data['grant_type'] = "client_credentials"
    auth_data['client_id'] = CLIENT_ID
    auth_data['client_secret'] = CLIENT_SECRET

    auth_response = requests.post(url=token_request_url, headers=auth_headers, data=auth_data)
    if auth_response.status_code == 200:
        print(json.dumps(auth_response.json(), indent=2)) 
        token = auth_response.json()['access_token']
    else:
        print('Error: Unable to obtain access token')
        return None
    
    url = 'https://api.spotify.com/v1/artists?ids='
    return_songs = songs
    temp_dict = artist_ids
    for idx,song in enumerate(songs):
        id = ''
        if song[1] not in temp_dict:
            id = get_artist_id(song[1])
            if id != '':
                temp_dict[song[1]] = id
                url += id
                if idx != len(songs) - 1:
                    url += '%2C'
            else:
                return_songs.remove(song)
        else:
            id = temp_dict[song[1]]
            url += id
            if idx != len(songs) - 1:
                url += '%2C'

    headers = {'Authorization': 'Bearer {token}'.format(token=token)}
    response = requests.get(url=url, headers=headers)
    if response.status_code == 200:
        artists = response.json()['artists']
        return_list = []
        print(len(artists))
        print(len(return_songs))
        if len(artists) != len(return_songs):
            print("return length different from input")
            return None
        else:
            for idx, artist in enumerate(artists):
                if artist:
                    return_list.append((return_songs[idx][0], return_songs[idx][1], artist['genres']))
                else:
                    print(f"artist nonetype at index {idx}")
            return return_list, temp_dict
    else:
        print(response.status_code)
        print('Error: Unable to fetch artist genre data')
        return None

In [5]:
def split_data(data, num_threads):
    chunk_size = len(data) // num_threads
    chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
    return chunks

In [6]:
def process_element(chunk):
    docs = []
    titles_artists = []
    english_words = None
    while english_words is None:
        try:
            english_words = set(words.words())
        except Exception as e:
            print(e)
            
    genius = Genius(genius_access_token, remove_section_headers=True)
    for track in chunk:
        
        title = track[0]
        feat_index = title.find("(feat.")
        with_index = title.find("(with")
        remastered_index = title.find("(Remastered")
        year_index = title.find("(2")

        if feat_index != -1:
            title = title[:feat_index]
        elif with_index != -1:
            title = title[:with_index]
        elif remastered_index != -1:
            title = title[:remastered_index]
        elif year_index != -1:
            title = title[:year_index]

        title = title.strip()
        try:
            song = genius.search_song(title, track[1])
        except Exception as e:
            print(type(e))
            print(e)
            continue
        lyrics = []

        if song == None:
            continue

        else:
            lyrics = song.lyrics.split('\n')[1:]
            if lyrics == []:
                continue
            last_line = lyrics[-1]
            pattern = r'\d.*'
            last_line = re.sub(pattern, '', last_line)
            lyrics[-1] = last_line
            new_lyrics = []
            dash_count = 0
            non_lyrics = False
            for line in lyrics:
                if dash_count >= 4:
                    non_lyrics = True
                    break
                if '-' in line:
                    dash_count += 1
                else:
                    dash_count = 0
                line = line.split()
                line = [word for word in line if word.lower() in english_words]
                if line != []:
                    line = ' '.join(line)
                    new_lyrics.append(line)
            if non_lyrics:
                continue
                
            filtered_lyrics = [line for line in new_lyrics if track[1] not in line]
            filtered_lyrics = [line for line in filtered_lyrics if '[' not in line and ']' not in line]
            
            if filtered_lyrics != []:
                doc = '\n'.join(filtered_lyrics)
                if not all(char.isspace() for char in doc):
                    doc = doc.translate(str.maketrans('', '', string.punctuation))
        #             doc = doc.lower()
                    doc = doc.replace('\n', ' ')
        #             words = word_tokenize(doc)
        #             stop_words = set(stopwords.words('english'))
        #             filtered_text = [word for word in words if word.lower() not in stop_words]
        #             filtered_text = ' '.join(filtered_text)
                    docs.append(doc)
                    titles_artists.append((track[0], track[1]))
    #   if len(docs) == 3:
    #     print(docs)
    #     print(titles_artists)
    #     break
    return docs, titles_artists

In [7]:
def process_data_multithreaded(data, num_threads):
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit the processing function for each chunk of data
        futures = [executor.submit(process_element, element) for element in data]

        # Store the returned lists from each process
        results = [future.result() for future in concurrent.futures.as_completed(futures)]

    # Combine the lists into a single list
    combined_list = []
    for result in results:
        combined_list.extend(result)

    return combined_list

In [8]:
chunks = split_data(data, 8)

In [ ]:
combined_list = process_data_multithreaded(chunks, 8)

'WordListCorpusReader' object has no attribute '_LazyCorpusLoader__reader_cls'
Searching for "Nobody Compares" by One Direction...
Searching for "Gorgeous Nightmare" by Escape the Fate...
Searching for "Looks Like We Made It" by Barry Manilow...
Searching for "Turn Me On" by Kevin Lyttle...
Searching for "Legendary" by YNW Melly...
Searching for "Where My Girls At" by 702...
Searching for "Fact" by Ghostluvme...
Searching for "Somebody to Shove" by Soul Asylum...
Done.
Searching for "Stick Baby" by Lil Gnar...
Done.
Searching for "February" by A Boogie Wit da Hoodie...
Done.
Searching for "The Most Beautiful Girl" by Charlie Rich...
Done.
Searching for "I\'ll Be There for You - Theme From 'Friends'" by The Rembrandts...
Done.
Searching for "Una Linda Mujer" by Nivel Codiciado...
Done.Done.
Searching for "Sink Water" by Hotel Ugly...
Done.
Searching for "Born Without a Heart" by Faouzia...

Searching for "Never Is Enough" by iann dior...
Done.
Searching for "One In The Chamber" by North

Done.Done.
Searching for "BIRDZ FEAT. RICK ROSS" by Denzel Curry...

Searching for "Throw Some D's" by Rich Boy...
Done.
Searching for "The Weight Of My Words - Four Tet Remix" by Kings of Convenience...
Done.
Searching for "Diggin' Up Bones" by Randy Travis...
Done.
Searching for "My Friends" by Ty Dolla $ign...
Done.
Searching for "Around the World" by Red Hot Chili Peppers...
Done.
Searching for "Classic Man" by Jidenna...
Done.Done.
Searching for "Neck & Wrist" by Pusha T...

Searching for "Arms Of A Stranger" by Niall Horan...
Done.
Searching for "From The Ritz To The Rubble" by Arctic Monkeys...
Done.
Searching for "Middle Of The Night" by The Vamps...
Done.
Searching for "Make Up Your Mind" by Cordae...
Done.
Searching for "Take Kare" by YNW Melly...
Done.
Searching for "South" by Galimatias...
Done.Done.
Searching for "Cold Heart - PS1 Remix" by Elton John...

Searching for "You're On" by Madeon...
Done.
Searching for "Gas Pedal - Remix" by Sage The Gemini...
Done.Done.
Searchi

Done.Done.
Searching for "Higher Power" by Coldplay...

Searching for "Leave Before The Lights Come On" by Arctic Monkeys...
Done.
Searching for "In Bloom" by ZEROBASEONE...
Done.
Searching for "Thot Shit" by Megan Thee Stallion...
Done.
Searching for "Alright" by Supergrass...
Done.
Searching for "Mandatory Suicide" by Slayer...
Done.Done.
Searching for "Chop My Money - Remix" by P-Square...

Searching for "Just A Fan" by Roar...
Done.
Searching for "Another Heartbreak" by Giveon...
Done.
Searching for "Blessings - Extended Version" by Big Sean...
Done.
Searching for "pouring" by iann dior...
Done.
Searching for "ME TENGO QUE IR" by KAROL G...
Done.Done.
Searching for "Cheerleader - Felix Jaehn Remix Radio Edit" by OMI...

Searching for "Be My Own Boyfriend" by Olivia Dean...
Done.
Searching for "Everything Happens For A Reason" by Madison Beer...
Done.
Searching for "White Light" by Gorillaz...
Done.
Searching for "How Many Licks?" by Lil' Kim...
Done.
Searching for "Lifestyle" by Dr

Done.
Searching for "Girls & Boys" by Good Charlotte...
Done.
Searching for "start again" by iann dior...
Done.
Searching for "High School Sweethearts" by Melanie Martinez...
Done.
Searching for "an Interlude Called “Circus”" by Saba...
Done.
Searching for "#PROUDCATOWNERREMIX" by XXXTENTACION...
Done.
Searching for "crash my whip" by iann dior...
Done.
Searching for "I'm Good Luv, Enjoy." by Aaron May...
Done.
Searching for "Lean Back" by Terror Squad...
Done.
Searching for "Lost" by Michael Bublé...
Done.
Searching for "Nightingale" by Demi Lovato...
Done.
Searching for "All We Ever Knew" by The Head And The Heart...
Done.
Searching for "Wiggle" by binki...
Done.
Searching for "505" by Arctic Monkeys...
Done.
Searching for "It Runs Through Me" by Tom Misch...
Done.
Searching for "Dance Me To The End Of Love" by Leonard Cohen...
Done.
Searching for "Venus In Furs" by The Velvet Underground...
Done.
Searching for "Freestyle 2" by Ken Carson...
Done.
Searching for "Us" by Regina Spektor

Done.
Searching for "In Ha Mood" by Ice Spice...
Done.
Searching for "Outta Control" by 50 Cent...
Done.
Searching for "All I Wanted" by Paramore...
Done.
Searching for "Hive" by Earl Sweatshirt...
Done.
Searching for "Survival Tactics" by Joey Bada$$...
Done.
Searching for "Boys Don't Cry" by The Cure...
Done.
Searching for "The Thrill Is Gone." by RAYE...
Done.
Searching for "Say You Won't Let Go" by James Arthur...
Done.
Searching for "Hero" by Skillet...
Done.
Searching for "Where You Lead" by Carole King...
Done.
Searching for "My Song" by Labi Siffre...
Done.
Searching for "The Girl From Ipanema" by Amy Winehouse...
Done.
Searching for "Love Like Blood" by Killing Joke...
Done.
Searching for "Everything To Everyone (Intro)" by Reneé Rapp...
Done.
Searching for "Rendezvous" by Jack Harlow...
Done.
Searching for "One Dance" by Drake...
Done.Done.
Searching for "Eat Your Young" by Hozier...

Searching for "Dinero" by Trinidad Cardona...
Done.
Searching for "Remember Me" by Jeremih..

...

Done.
Searching for "Peace of Mind" by Boston...
Done.
Searching for "Rock Bottom" by Eminem...
Done.
Searching for "alotta cake" by Gunna...
Done.
Searching for "Carrollton" by $uicideboy$...
Done.
Searching for "Everyone's An Outlaw" by Bryan Martin...
Done.
Searching for "Mary Jane's Last Dance" by Tom Petty and the Heartbreakers...
Done.
Searching for "hue_man nature" by Saba...
Done.
Searching for "Cinema" by Harry Styles...
Done.
Searching for "Drunk On You" by Luke Bryan...
Done.
Searching for "Gone Daddy Gone" by Violent Femmes...
Done.
Searching for "Grow as We Go" by Ben Platt...
Done.
Searching for "Numb" by Linkin Park...
Done.
Searching for "Ain't It Different" by Headie One...
Done.
Searching for "Wild Thoughts" by DJ Khaled...
Done.
Searching for "Hold You Down" by The Alchemist...
Done.
Searching for "Daydreamer" by Adele...
Done.
Searching for "REDMERCEDES" by Aminé...
Done.
Searching for "Watermelon Crawl" by Tracy Byrd...
Done.
Searching for "Leaving You" by Riovaz..

Done.
Searching for "Lamento Boliviano" by Los Enanitos Verdes...
Done.
Searching for "DEVIL" by Shinedown...
Done.
Searching for "Yea Yea" by Pop Smoke...
Done.
Searching for "Mad At Me." by Kiana Ledé...
Done.
Searching for "A Ray of Sunshine" by Wham!...
Done.
Searching for "Compensating" by Aminé...
Done.
Searching for "El Gordo Trae El Mando" by Chino Pacas...
Done.
Searching for "Bruuuh" by JID...
Done.
Searching for "Sweet Little Lies" by Rod Wave...
Done.
Searching for "Carry On" by fun....
Done.
Searching for "Old Friend" by Earl Sweatshirt...
Done.
Searching for "Hello" by Adele...
Done.
Searching for "Enchanted" by Taylor Swift...
Done.
Searching for "Knock Knock" by Mac Miller...
Done.
Searching for "Roygbiv" by Boards of Canada...
Done.
Searching for "I Wouldn't Ask You" by Clairo...
Done.
Searching for "Simmer" by Mahalia...
Done.
Searching for "Ain't Livin Right" by Future...
Done.
Searching for "Homage" by Mild High Club...
Done.
Searching for "Sorry" by Halsey...
Done.

Done.
Searching for "Today" by Cordae...
Done.
Searching for "Sacrifices" by Drake...
Done.Done.
Searching for "Light It Up - Remix" by Major Lazer...

Searching for "Love Is Madness" by Thirty Seconds To Mars...
Done.
Searching for "Switched Up" by Morray...
Done.
Searching for "Ella" by Junior H...
Done.
Searching for "Miracles" by Coldplay...
Done.Done.
Searching for "2 Grown" by Lil Tjay...

Searching for "Don’t Play" by Halsey...
Done.
Searching for "Cool Kids" by Echosmith...
Done.
Searching for "My Universe" by Coldplay...
Done.Done.
Searching for "Paramedic!" by SOB X RBE...

Searching for "By Design" by Kid Cudi...
Done.
Searching for "Only The Young" by Brandon Flowers...
Done.
Searching for "Electric Love" by BØRNS...
Done.
Searching for "Lucidity" by Tame Impala...
Done.
Searching for "The Scientist" by Coldplay...
Done.
Searching for "Link Up - Spider-Verse Remix (Spider-Man: Across the Spider-Verse )" by Metro Boomin...
Done.
Searching for "Sidewalks" by The Weeknd...
Don

Done.
Searching for "XO" by Eden Project...
Done.
Searching for "The Diary of Jane - Single Version" by Breaking Benjamin...
Done.
Searching for "You Proof" by Morgan Wallen...
Done.
Searching for "Kitchen Kings" by D-Block Europe...
Done.
Searching for "What Would I Do?" by Strawberry Guy...
Done.
Searching for "Mona Lisa" by Lil Wayne...
Done.
Searching for "Time to Pretend" by MGMT...
Done.
Searching for "We Gonna Take U Back (Lude) / Don't Leave Me" by Blackstreet...
Done.
Searching for "Come Get It Bae" by Pharrell Williams...
Done.
Searching for "Carry On" by Avenged Sevenfold...
Done.
Searching for "The Way I Am" by Eminem...
Done.
Searching for "New" by Daya...
Done.
Searching for "Rock 'N' Roll All Nite - Live" by KISS...
Done.
Searching for "Fake Plastic Trees" by Radiohead...
Done.
Searching for "I Did It" by A Boogie Wit da Hoodie...
Done.
Searching for "More Than My Hometown" by Morgan Wallen...
Done.
Searching for "4r Da Squaw" by Isaiah Rashad...
Done.
Searching for "Eve

Done.
Searching for "Been Thinking" by Tyla...
Done.
Searching for "Keep on Loving You" by REO Speedwagon...
Done.
Searching for "If I Ruled the World (Imagine That)" by Nas...
Done.
Searching for "The Sound" by The 1975...
Done.
Searching for "Want Her" by Mustard...
Done.
Searching for "What You Know" by T.I....
Done.
Searching for "family ties" by Baby Keem...
Done.
Searching for "killer" by FKA twigs...
Done.
Searching for "Interlude : Dive" by Jimin...
<class 'requests.exceptions.Timeout'>
Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)
Searching for "하루살이" by DEAN...
Done.
Searching for "Players - DJ Smallz 732 - Jersey Club Remix" by Coi Leray...
Done.
Searching for "It's You" by Ali Gatie...
Done.
Searching for "Brilliant Bitch" by SAINt JHN...
Done.
Searching for "Protect Ya Neck" by Wu-Tang Clan...
Done.
Searching for "When Emma Falls in Love (Taylor’s Version) (From The Vault)" by Taylor Swift...
Done.
Searching for "Decl

In [ ]:
combined_docs = []
combined_songlabels = []
counter = 0
for piece in combined_list:
    if counter % 2 == 0:
        combined_docs.extend(piece)
    else:
        combined_songlabels.extend(piece)
    counter += 1

In [ ]:
print(len(combined_docs))
print(len(combined_songlabels))
print(combined_songlabels)

In [ ]:
import os
path = 'docs'
for idx, doc in enumerate(combined_docs):
    filename = f'{combined_songlabels[idx][1]}_{combined_songlabels[idx][0]}.txt'
    with open(os.path.join(path, filename), 'w') as doc_file:
        doc_file.write(combined_songlabels[idx][0]+' by '+combined_songlabels[idx][1]+"\n\n"+doc)